<a href="https://colab.research.google.com/github/Lusiji254/Pneumothorax-Segmentation-using-DeepLab/blob/main/VGG19_Classification_Model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
from glob import glob

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.applications import VGG19
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [ ]:
def build_model(size, num_classes):
    inputs = Input((size, size, 3))
    backbone = VGG19(input_tensor=inputs, include_top=False, weights="imagenet")
    backbone.trainable = False
    x = backbone.output
    x = GlobalAveragePooling2D()(x.layers[-3].output)
    x = Dropout(0.2)(x)
    x = Dense(1024, activation="relu")(x)
    x = Dense(num_classes, activation="sigmoid")(x)

    model = tf.keras.Model(inputs, x)
    return model


In [ ]:
def read_image(path, size):
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (size, size))
    image = image / 255.0
    image = image.astype(np.float32)
    return image

def parse_data(x, y):
    x = x.decode()

    num_class = 2
    size = 224

    image = read_image(x, size)
    label = [0] * num_class
    label[y] = 1
    label = np.array(label)
    label = label.astype(np.int32)

    return image, label

def tf_parse(x, y):
    x, y = tf.numpy_function(parse_data, [x, y], [tf.float32, tf.int32])
    x.set_shape((224, 224, 3))
    y.set_shape((2))
    return x, y

def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    return dataset


In [ ]:
import keras.backend as K
def f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
if __name__ == "__main__":
    path = "/content/gdrive/MyDrive/Pneumothorax"
    train_path = os.path.join(path, "downsampled_train_images/*")
    test_path = os.path.join(path, "test_images/*")
    labels_path = os.path.join(path, "downsampled_train.csv")

    labels_df = pd.read_csv(labels_path)
    Pneumothorax = labels_df["has_pneumo"].unique()
    print("Number of Classes: ", len(Pneumothorax))

    ids = glob(train_path)
    labels = []

    for image_name in ids:
        image_name = image_name.split("/")[-1]
        image_with_class = list(labels_df[labels_df.new_filename == image_name]["has_pneumo"])[0]
        labels.append(image_with_class)

    train_x, valid_x = train_test_split(ids, test_size=0.2, random_state=42)
    train_y, valid_y = train_test_split(labels, test_size=0.2, random_state=42)
 


   ## Parameters
    size = 256
    num_classes = 2
    lr = 1e-3
    batch = 16
    epochs = 100
    model_path = os.path.join("/content/gdrive/MyDrive/Pneumothorax/pneumo-classify", "classification_model_tbrl.h5")

    ## Model
    model = build_model(size, num_classes)
    adam = tf.keras.optimizers.Adam(lr=0.001)
    precision = tf.keras.metrics.Precision(name='precision')
    recall = tf.keras.metrics.Recall(name='recall')

    model.compile(loss="binary_crossentropy", optimizer=Adam(lr), metrics=["accuracy",f1, recall, precision])


    ## Dataset
    train_dataset = tf_dataset(train_x, train_y, batch=batch)
    valid_dataset = tf_dataset(valid_x, valid_y, batch=batch)

     ## Training
    %rm -rf ./log
    %load_ext tensorboard
    %tensorboard --logdir='/content/gdrive/MyDrive/Pneumothorax/Tensorboard'
    %reload_ext tensorboard
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='/content/gdrive/MyDrive/Pneumothorax/Tensorboard')
    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False),
        tensorboard_callback
    ]
    train_steps = (len(train_x)//batch) + 1
    valid_steps = (len(valid_x)//batch) + 1
    model.fit(train_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        validation_data=valid_dataset,
        epochs=epochs,
        callbacks=callbacks)




In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt 
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score



H = 256
W = 256


def read_image(path, size):
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (size, size))
    image = image / 255.0
    image = image.astype(np.float32)
    return image

if __name__ == "__main__":
    path = "/content/gdrive/MyDrive/Pneumothorax"
    test_path = os.path.join(path, "test_images/*")
    labels_path = os.path.join(path, "stage_1_test_images.csv")

    labels_df = pd.read_csv(labels_path)
    Pneumothorax = labels_df["has_pneumo"].unique()
    print("Number of Classes: ", len(Pneumothorax))
   
    ids = glob(test_path)
    labels = []
     
    for image_name in ids:
        image_name = image_name.split("/")[-1]
        image_with_class = list(labels_df[labels_df.new_filename == image_name]["has_pneumo"])[0]
        labels.append(image_with_class)

    test_x = ids[:200]
    test_y = labels[:200]  

    ## Model
    with CustomObjectScope({'f1':f1}):
        model_path = os.path.join("/content/gdrive/MyDrive/Pneumothorax/pneumo-classify", "chexnet_model.h5")
        model = tf.keras.models.load_model(model_path)

    for i, path in tqdm(enumerate(test_x[:10])):
          image = read_image(path, 256)
          image = np.expand_dims(image, axis=0)
          print(image.shape)
          y_pred = model.evaluate(image)[0]
          y_pred = np.argmax(y_pred)
          
          ori_pneumo = [test_y[i]]
          ori_image = cv2.imread(path, cv2.IMREAD_COLOR)

          f, (ax1) = plt.subplots(1, 1, figsize=(10,5))
                  
          ax1.set_title(f'IMAGE. The initial class was: {ori_pneumo}, the prediction is: {y_pred}')
          ax1.imshow(image[0])
